In [1]:
import os

In [2]:
import pandas as pd

In [3]:
METHODS = ['default', 'ror','uniform','nri_uniform','var']
DATA_LBP_PATH = 'dataset/fish_data_lbp/resize/'
DATA_HOG_PATH = 'dataset/fish_data_hog'
DATA_HL_PATH = 'dataset/fish_data_houghline'
METHOD_INDEX = 4


In [4]:
METHOD_INDEX = 4
# df = pd.read_csv(os.path.join(DATA_LBP_PATH,METHODS[METHOD_INDEX],METHODS[METHOD_INDEX]+'_dense_radius_is_3.csv'), header=None)
df = pd.read_csv(os.path.join(DATA_HOG_PATH,'hog_fish.csv'), header=None)
df.head(20)

,0,1,2,3,4,5,6,7,8,9,...,4755,4756,4757,4758,4759,4760,4761,4762,4763,4764
0,1,A73EGS-P,1,0.039008,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1,2,A73EGS-P,2,0.027035,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.402458,0.252968,0.402458,0.301150,0.402458,0.036988,0.329734,0.295521,0.402458
2,3,A73EGS-P,3,0.025006,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
3,4,A73EGS-P,4,0.035968,0.414215,0.414215,0.414215,0.351986,0.414215,0.120783,...,0.082304,0.432131,0.430995,0.432131,0.112794,0.432131,0.186883,0.432131,0.046721,0.132146
4,5,A73EGS-P,5,0.027966,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
5,6,A73EGS-P,6,0.032001,0.401761,0.125557,0.401761,0.125557,0.401761,0.401761,...,0.000000,0.526888,0.000000,0.456013,0.120170,0.526888,0.120170,0.456013,0.000000,0.000000
6,7,A73EGS-P,7,0.035001,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
7,8,acanthaluteres_brownii,1,0.025000,0.446425,0.411431,0.446425,0.235103,0.446425,0.083122,...,0.386499,0.376669,0.144562,0.327824,0.376669,0.156876,0.376669,0.370978,0.376669,0.376669
8,9,acanthaluteres_brownii,2,0.033000,0.444278,0.316934,0.392861,0.056470,0.444278,0.112940,...,0.000000,0.601388,0.199759,0.084226,0.000000,0.238227,0.066586,0.601388,0.399518,0.094167
9,10,acanthaluteres_brownii,3,0.029996,0.342638,0.342638,0.246561,0.342638,0.342638,0.342638,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


In [5]:
from sklearn import preprocessing
lb = preprocessing.LabelEncoder()
lb.fit(df[1].unique())

LabelEncoder()

In [6]:
new_class = lb.transform(df[1])
new_class
# df.head()


array([  0,   0,   0, ..., 483, 483, 483])

In [7]:
# from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split

In [8]:
index_input =[i for i in range(4,260)]
X_train, X_test, y_train, y_test = train_test_split(df[index_input].copy().to_numpy(), new_class, 
                                                    test_size=0.333,random_state=1,stratify=new_class)

In [9]:
y_test

array([348, 144, 417, ..., 460,  41, 100])

In [10]:
import torch as tr
import torch.nn as tr_nn
import torch.nn.functional as tr_fn
import torch.optim as tr_optim
# import torchvision
# import torchvision.transforms as tr_transforms


In [11]:
device = 'cpu'
if(tr.cuda.is_available()):
    device = tr.device('cuda')    
print(device)


cuda


In [12]:
x_test_tr = tr.from_numpy(X_test).view(X_test.shape[0],256).type(tr.float)
x_train_tr = tr.from_numpy(X_train).view(X_train.shape[0],256).type(tr.float)
y_test_tr = tr.from_numpy(y_test).view(-1,1).type(tr.float)
y_train_tr = tr.from_numpy(y_train).view(-1,1).type(tr.float)
# print(x_train_tr.shape, y_train_tr)

# x_test_tr = tr.randn(X_test.shape[0], 20)
# x_train_tr = tr.randn(X_train.shape[0], 20)
# y_test_tr = tr.from_numpy(y_test).view(-1,1).type(tr.float)
# y_train_tr = tr.from_numpy(y_train).view(-1,1).type(tr.float)
print(x_test_tr.shape)
print(x_test_tr)

torch.Size([1471, 256])
tensor([[0.5473, 0.0000, 0.5473,  ..., 0.4798, 0.2425, 0.0767],
        [0.6837, 0.1666, 0.0329,  ..., 0.5609, 0.0916, 0.1278],
        [0.4688, 0.2221, 0.4688,  ..., 0.3865, 0.3112, 0.3460],
        ...,
        [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
        [0.5203, 0.1312, 0.5203,  ..., 0.1293, 0.5724, 0.0000],
        [0.4383, 0.3574, 0.3032,  ..., 0.3844, 0.3844, 0.3128]])


In [13]:
import math
batch_size = 300
rows_size = X_train.shape[0]
batch_rows = rows_size/batch_size
print(batch_rows)
batch_epoch = math.ceil(batch_rows)
print(batch_epoch)

9.813333333333333
10


In [22]:
class Net(tr_nn.Module):
    def __init__(self,input,output):
        super(Net, self).__init__()
        self.conv1 = tr_nn.Linear(input, 5000)
        self.pool = tr_nn.MaxPool1d(3, 1)
        self.conv2 = tr_nn.Linear(4998, 2000)
        self.fc1 = tr_nn.Linear(2000, 1020)
        self.fc2 = tr_nn.Linear(1020, 600)
        self.fc3 = tr_nn.Linear(600, output)
        self.dp = tr_nn.Dropout(0.2)

    def forward(self, x):
        x = self.pool(self.conv1(x))
        x = self.conv2(x)
        x = self.dp(x)
        # x = tr.flatten(x, 1) # flatten all dimensions except batch
        x = tr_fn.relu(self.fc1(x))
        x = self.dp(x)
        x = tr_fn.relu(self.fc2(x))
        x = self.fc3(x)
        return x


In [23]:
model = Net(x_test_tr.shape[1],len(df[1].unique())).to(device)
model

Net(
  (conv1): Linear(in_features=256, out_features=5000, bias=True)
  (pool): MaxPool1d(kernel_size=3, stride=1, padding=0, dilation=1, ceil_mode=False)
  (conv2): Linear(in_features=4998, out_features=2000, bias=True)
  (fc1): Linear(in_features=2000, out_features=1020, bias=True)
  (fc2): Linear(in_features=1020, out_features=600, bias=True)
  (fc3): Linear(in_features=600, out_features=484, bias=True)
  (dp): Dropout(p=0.2, inplace=False)
)

In [24]:

# criterion = tr_nn.BCELoss()
# optimizer = tr_optim.Adam(model.parameters(), lr=0.001)
criterion = tr_nn.CrossEntropyLoss()
optimizer = tr_optim.Adam(model.parameters(), lr=0.001)

In [25]:


for epoch in range(2000):  # loop over the dataset multiple times
    running_loss = 0.0
    for i in range(0,batch_epoch):
        # get the inputs; data is a list of [inputs, labels]
        endp = (i+1)*batch_size
        inputs = x_train_tr[i*batch_size:endp if endp < rows_size else rows_size,:]
        labels = y_train_tr [i*batch_size:endp if endp < rows_size else rows_size]
        labels = labels.to(device)
        inputs = inputs.to(device)
        # zero the parameter gradients
        optimizer.zero_grad()
        # print("inuts", inputs)
        # print("labels",labels)
        # forward + backward + optimize
        outputs = model(inputs)
        # print("outputs",outputs)
        labels = labels.squeeze(1).type(tr.LongTensor).to(device)
        # print("label seq",labels)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        # # print statistics
        running_loss += loss.item()
        if epoch % 10 == 0:    # print every 2000 mini-batches
            print(f'[{epoch + 1}, {i + 1:5d}] loss: {running_loss:.6f}')
        running_loss = 0.0
    if epoch % 10 ==0:
         with tr.no_grad():
                y_test_tr_tr = y_test_tr.to(device).squeeze(1).type(tr.int)
                y_pr = model(x_test_tr.to(device))
                _, y_pr_class = tr.max(y_pr,1)
                acc = (y_pr_class==y_test_tr_tr).sum().float()/float(y_test_tr_tr.shape[0])
                print(f"[{epoch + 1}] acc {acc:.3f}")
    
    
    
print('Finished Training')

[1,     1] loss: 6.181760
[1,     2] loss: 6.194138
[1,     3] loss: 6.224454
[1,     4] loss: 6.223843
[1,     5] loss: 6.176038
[1,     6] loss: 6.150723
[1,     7] loss: 6.170039
[1,     8] loss: 6.141735
[1,     9] loss: 6.139662
[1,    10] loss: 6.171342
[1] acc 0.008
[11,     1] loss: 4.136279
[11,     2] loss: 3.795315
[11,     3] loss: 3.975297
[11,     4] loss: 4.253778
[11,     5] loss: 3.984774
[11,     6] loss: 4.137417
[11,     7] loss: 4.105026
[11,     8] loss: 3.874306
[11,     9] loss: 3.843989
[11,    10] loss: 3.826778
[11] acc 0.052
[21,     1] loss: 2.407529
[21,     2] loss: 2.422088
[21,     3] loss: 2.306245
[21,     4] loss: 2.181573
[21,     5] loss: 2.458708
[21,     6] loss: 2.278996
[21,     7] loss: 2.400770
[21,     8] loss: 2.225919
[21,     9] loss: 2.039299
[21,    10] loss: 1.963828
[21] acc 0.061
[31,     1] loss: 0.929424
[31,     2] loss: 1.048585
[31,     3] loss: 0.774660
[31,     4] loss: 0.818112
[31,     5] loss: 0.748497
[31,     6] loss: 0.9

KeyboardInterrupt: 

In [115]:
with tr.no_grad():
    y_test_tr_tr = y_test_tr.to(device).squeeze(1).type(tr.int)
    y_pr = model(x_test_tr.to(device))
    _, y_pr_class = tr.max(y_pr,1)
    
    print(y_pr_class,y_test_tr_tr)
    acc = (y_pr_class==y_test_tr_tr).sum().float()/float(y_test_tr_tr.shape[0])
    # for i in range(0,y_pr_class.shape[0]):
    #     print(y_pr_class[i],y_test_tr[i])
    print("acc", acc)

tensor([78, 78, 78,  ..., 78, 78, 78], device='cuda:0') tensor([162,  63, 130,  ..., 157, 110, 158], device='cuda:0',
       dtype=torch.int32)
acc tensor(0.0027, device='cuda:0')


In [63]:
conv1 = tr_nn.Conv1d(256, 200, 1)
pool = tr_nn.MaxPool1d(1)
conv2 = tr_nn.Conv1d(200, 16, 1)
fc1 = tr_nn.Linear(16, 120)
fc2 = tr_nn.Linear(120, 84)
fc3 = tr_nn.Linear(84, 10)

def tt(x):
    x = pool(tr_fn.relu(conv1(x)))
    x = pool(tr_fn.relu(conv2(x)))
    x = tr.flatten(x, 1) # flatten all dimensions except batch
    x = tr_fn.relu(fc1(x))
    x = tr_fn.relu(fc2(x))
    x = fc3(x)
    return x
# input = tr.randn(20, 15, 50)
# print(input[2].shape)
# print(m)
yyy = x_test_tr[i*batch_size:(i+1)*batch_size,:]
output = tt(yyy)
output

RuntimeError: Expected 3-dimensional input for 3-dimensional weight [200, 256, 1], but got 2-dimensional input of size [0, 20] instead